In [1]:
from bs4 import BeautifulSoup
import requests
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

import torch as t
from torch.utils import data
from torchvision import transforms as tsf
import scipy.misc
import torch
from pathlib import Path



from PIL import Image
import skimage
from skimage import io

In [2]:
meme_links = list()
meme_links.append('/meme/First-World-Problems?sort=top-2019')
meme_links.append('/meme/Grandma-Finds-The-Internet?sort=top-2019')
meme_links.append('/meme/Success-Kid?sort=top-2019')
meme_links.append('/meme/Leonardo-Dicaprio-Cheers?sort=top-2019')
meme_links.append('/meme/That-Would-Be-Great?sort=top-2019')
meme_links.append('/meme/Creepy-Condescending-Wonka?sort=top-2019')
meme_links.append('/meme/Disaster-Girl?sort=top-2019')
meme_links.append('/meme/Futurama-Fry?sort=top-2019')
meme_links.append('/meme/One-Does-Not-Simply?sort=top-2019')

In [3]:
image_names = ['source_memes/'+'First-World-Problems.jpg','source_memes/Grandma-Finds-The-Internet.jpg','source_memes/Success-Kid.jpg',
              'source_memes/'+'Leonardo-Dicaprio-Cheers.jpg','source_memes/That-Would-Be-Great.jpg','source_memes/'+'Creepy-Condescending-Wonka.jpg',
              'source_memes/'+'Disaster-Girl.jpg','source_memes/'+'Futurama-Fry.jpg','source_memes/'+'One-Does-Not-Simply.jpg']

In [4]:
images = []
for name in image_names:
    input_image = Image.open(name)
    preprocess = tsf.Compose([
        tsf.ToTensor(),
        tsf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0) 
    
    images.append(input_batch)

In [5]:
#mapping meme name to image
desc =list()
desc.append(('First World Problems',images[0]))
desc.append(('Grandma Finds The Internet',images[1]))
desc.append(('Success Kid',images[2]))
desc.append(('This is to all',images[3]))
desc.append(('That would be great',images[4]))
desc.append(('Creepy Condescending Wonka',images[5]))
desc.append(('Disaster Girl',images[6]))
desc.append(("NOT SURE IF",images[7]))
desc.append(("ONE DOES NOT SIMPLY",images[8]))


In [ ]:
#parsing

datasets = list()
i = 0
for link in meme_links:
    dataset = list()

    while link is not None:
        html_1 = requests.get('https://imgflip.com'+link)
        if i % 10 == 0:
            print('{} links proccessed'.format(i))
        i += 1
        if html_1.status_code != 200:
            print('ERROR')
            break
        soup = BeautifulSoup(html_1.text, 'html.parser')
        lef = soup.find("div", {"id": "base-left"})
        img = lef.find_all('img')
        units = lef.find_all(class_ ='base-unit clearfix')
        for u in units:
            if u.find('img')  and u.find('img')['alt'] != '':
                alt_tag = u.find('img')['alt'].split('|')[1].strip()
                dataset.append(alt_tag)
        link = soup.find(class_ = 'pager-next l but')
        if link:
            link = link['href']

    datasets.append(dataset)

In [ ]:
pic_id = 0
lines = list()
for pic_id, dataset in enumerate(datasets):
    for text in dataset:
        print(text)
        lines.append((pic_id, text))

In [6]:
import numpy as np
import re
from torch import nn
from torch.autograd import Variable
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import sys
from IPython.display import clear_output
import tqdm
import progressbar

import torchvision
import torch
import pickle
%matplotlib inline

In [7]:
all_words = set()
with open('glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        if line[0] not in all_words:
            all_words.add(line[0])
        

In [ ]:
new_lines = list()
for line in lines:
    line1 = re.sub(r'[^; \w]+', '', line[1])
    k = line1.find(';')
    if k > 0:
        line1 = line1[:k] +' ' + line1[k:]
    fl = 0
    for k in line1.split(' '):
        if k.lower() not in all_words:
            fl = 1
    if not fl:
        new_lines.append((line[0],line1))   

In [ ]:
lines = new_lines

In [8]:
class Vocabulary:
    def __init__(self, sentences):
        self.all_characters = set()
        for line in sentences:
            line = re.sub(r'[^; \w]+', '', line[1])
            print(line)
            for k in line.split(' '):
                if k.lower() not in self.all_characters:
                    self.all_characters.add(k.lower())
        self.all_characters = list(sorted(self.all_characters))+['<eos>', '<go>']
        self.char_to_id = {x[1]:x[0] for x in enumerate(self.all_characters)}
        self.id_to_char = {x[0]:x[1] for x in enumerate(self.all_characters)}
        self.size = len(self.all_characters)

    def encode(self, line):
        return [self.char_to_id[x] for x in line]
    
    def decode(self, tokens):
        return ''.join([self.id_to_char[x] for x in tokens])

In [9]:
import pickle
with open('linesREALTOPnew3.pickle', 'rb') as f:
    lines =  pickle.load(f) 

In [10]:
vocab = Vocabulary(lines)

I SPENT 10 MINUTES LOOKING FOR MY PHONE IN MY CAR ; WHILE USING MY PHONE AS THE FLASHLIGHT
8 MONTHS PREGNANT AND TOLD MY HUSBAND TO PUT THE OREOS SOMEWHERE I COULDNT REACH THEM ; HE PUT THEM ON THE FLOOR
DAY 12 WITHOUT CHOCOLATE ; LOSING HEARING IN MY RIGHT EYE
I HAD 3 HAPPY MEALS YESTERDAY ; BUT NONE OF THEM WORKED
FOUND FAVORITE FOOD AT END OF BUFFET ; PLATE ALREADY FULL
I ACCIDENTALLY WORE A RED SHIRT TO TARGET THIS MORNING ; NOW IM COVERING FOR DEBBIE THIS WEEKEND
MY FAVORITE PART OF GROCERY SHOPPING WITH KIDS ; IS STAYING HOME AND STARVING INSTEAD
Ive decided not to have kids ; My kids are taking it pretty hard
THE BOX OF ANIMAL CRACKERS SAID DO NOT EAT IF THE SEAL IS BROKEN I OPENED THE BOX AND SURE ENOUGH
HOW IS IT POSSIBLE FOR A 2 POUND BOX OF CHOCOLATES ; TO MAKE YOU 5 POUNDS HEAVIER
BIKINI SEASON IS JUST AROUND THE CORNER ; UNFORTUNATELY SO IS THE TACO TRUCK
I JUST WANT MY HOUSE TO BE CLEAN ENOUGH SO THAT IF SOMEONE DROPS BY UNEXPECTEDLY ; IT DOESNT LOOK LIKE WERE SIX DAYS IN

ASKS HUSBAND TO WALK IN MY SHOES FOR A DAY ; NOW HES IGNORING ME AND I DONT KNOW WHY
PEOPLE WHO KNOW MEMES DONT LIKE SCIENTOLOGY ; AND SCIENTOLOGISTS CANT LOOK AT MEMES
WHENEVER I GO TO A FAMILY REUNION ; I CANT HELP CONSIDERING GETTING MY TUBES TIED
WHEN YOURE STUCK ; BETWEEN BIRTH AND DEATH
I CANT SEE ; MY SHADOW WITH MY EYES CLOSED
WHEN YOU FORGOT TO DO YOUR HOMEWORK BECAUSE YOU WERE MAKING MEMES
ME DAD A STRANGER IS CHASING ME ; DAD FATHER TOBIN
WHEN YOU MISS PLACE A GLASS BLOCK
WHEN DONALD TRUMP SAID ; WHITES ONLY
WHEN NO ONE LIKES YOUR MEMES ANYMORE
WHEN THEY SAY THEY HAVE DANK MEMES
WHEN YOU FEEL BAD FOR NOT HELPING MORE POOR PEOPLE NO WAIT YOU DONT EVEN FEEL BAD ANYMORE
GET YOUR BSN THEY SAID ; ITS NOT THAT BAD THEY SAID
WHEN U TRY UR BEST ; AND YOU DONT SUCCEED
THE WITNESSES CAME TO MY DOOR I JOINED
THE PROS OF COOKING THE FOOD ; THE CONS OF COOKING THE COOKING
MY TINDER DATE ; DOESNT THINK IM WOKE
BE CAREFUL ; WHAT YOU WISH FOR
LOS ANGELES NEEDS THIS RAIN ; BUT MY SOLAR PANEL

THANK YOU SIR
HERES TO ALL THE FATHERS IN THE WORLD BECAUSE WITHOUT THEM ; YOU MOM WOULDNT HAVE HER DAY EITHER
CHEERS TO PARENTS OF GENERATION Z ; BECAUSE YOU RAISED YOUR KIDS TO GET THEIR ASSES KICKED IN THE REAL WORLD
HERES TO ALL THE PEOPLE ; THAT DONT SPOIL AVENGERS ENDGAME FOR OTHERS
HERES TO YOU MY FRIEND ; WAY TO SCREW UP A PERFECTLY GOOD LIFE
WHEN YOUR DAD BEATS YOU ; BUT YOU GOT VICTORY ROYAL
WHEN YOU MAKE A GOOD MEME ; AND IT ACTUALLY GETS SOME ATTENTION
HERES TO THE REAL WEALTH CREATORS THE FARMERS RANCHERS MINERS LOGGERS AND FISHERMEN THAT HARVEST NATURAL RESOURCES AND MAKE A DOLLAR THAT WAS NOT THERE BEFORE
TO ALL THE PEOPLE WHO JUST WANT TO BE THEMSELVES AND DONT CARE ABOUT FITTING IN
HAPPY VALENTINES DAY ; AKA NATIONAL SINGLES AWARENESS DAY
THANK YOU ALL FOR 20000 POINTS
CHEERS TO ALL OF YOU ; LONELY LIFELESS BEINGS WHO SPEND ALL YOUR TIME STARING AT OTHER PEOPLES MEMES
HAPPY NEW YEAR
HERES TO A NEW YEARS EVE ; OF THE 2019
HERES TO THE TWENTIES
Never say New Year New Me 

YESTERDAY I BUMPED INTO THE GUY THAT SOLD ME AN ANTIQUE GLOBE ; ITS A SMALL WORLD
SO YOU THINK A GUN BAN WILL MAKE THE WORLD BETTER TELL ME AGAIN HOW CRIMINALS FOLLOW LAWS
JUST LOOK ON THE POSITIVE SIDE YOUR SCHIZOPHRENIA BEATS BEING ALONE
IF VEGETABLES ARE SO GOOD WHY DO VEGANS ALWAYS TRY TO MAKE THEM TASTE LIKE MEAT
GUN LAWS HUH ; TELL ME WHEN HAS A CRIMINAL EVER FOLLOWED A LAW
OH IM SORRY THAT THE MIDDLE OF MY SENTENCE ; INTERRUPTED THE BEGINNING OF YOURS
DYSLEXIC YOU SAY HOW DO YOU SPELL THAT
I AM GLAD TO SEE THAT YOU HAVE NEVER LET YOUR ; ADVANCED EDUCATION GET IN THE WAY OF YOUR IGNORANCE
HAVING A PET IS LIKE HAVING A CHILD HUH REMIND ME AGAIN HOW LONG WERE YOU IN LABOR WITH YOUR TARANTULA
A SNOBBISH CRIMINAL GOING DOWN THE STAIRS CONDESCENDING CON DESCENDING
YOU THINK HAVING A KID MAKES YOU A PARENT DOES HAVING A PIANO MAKE YOU A PIANIST
ME TRYING TO CONVINCE YOU TO UP VOTE THIS ; YOURE AN AMAZING HUMAN BEING ;
WE SHOULD REALLY HONOR AND BUILD A STATUE TO THE ; HERO THAT KILLED 

WHEN YOUR MOM AND DAD ; DONT GIVE YOU DESSERT
HE BROKE UP HIS RELATIONSHIP WITH ME ; I BROKE UP HIS RELATIONSHIP WITH LIFE ITSELF
MY WORK HERE ; IS ALMOST DONE
YOUTUBE IN 2020 ; COPPA
I PUT NUTELLA IN THE MICROWAVE ; EVERYONE
WE COOKED OUR BIRD IN THE DEEP FRYER ; AND I HELPED
THIS IS WHY ; WE HIRE BABYSITTERS
THEY SAID NO COOKIES ; THERE REGRETTING IT NOW
HOUSE BURNS DOWN ; NOBODY NOTICED
EVERYTHING IN MY LIFE GOING WRONG ; ME
I WATCHED THE NOTRE DAME FIRE AND GOT INSPIRED ; TO SOON
ME WHEN I CANT BEAT THAT ONE LEVEL ON ; GEOMETRY DASH
NO MORE HOMEWORK ; FOR ME
TOLD MOM I WAS MAKING PIZZA ROLLS ; I HAD A BLAST
I SAW A BUG ; NOW ITS KILLED
MY MOM ALWAYS TOLD ME TO BE ADVENTUROUS ; SHELL BE SO PROUD WHEN SHE SEES THIS
WHEN A SPIDER BE THINKING I AINT SERIOUS
MUMMY SAID I COULDNT PLAY WITH MY BARBIES ; TOO BAD
THE FACE YOU MAKE ; AFTER BURNING YOUR XS HOUSE
I ONLY GOT 167 TREATS YOU SHOULD HAVE GOTTEN ME 168
WHEN YOUR MOM SAYS ; YOU COULD DO WHATEVER YOU WANT
THANOS ; THOR
BURNING CALORI

SAVAGE
HE DIDNT GIVE ME A CHRISTMAS PRESENT ; SO I GAVE HIM ONE
WHEN YOUR PARENTS ; DONT LET YOU PLAY MINECRAFT
MY HUSBAND COMPLAINED THE FOOD WAS COLD IS THIS WARM ENOUGH
GETS KILLED IN TOP 2 BY A NOOB ; HE HAS BEEN DEFEATED
STEALING GAMES ; FROM STEAM
THEY DIDNT LIKE THRIVE
LIKE I TOLD YOU I HAVENT BRUSHED ; BECAUSE WERE OUT OF TOOTHPASTE
MY TEACHER TOLD ME TO DO MY HOMEWORK ; BUT YOU NEED A HOME FOR THAT
I TOLD YOU THERE WAS A SPIDER ; AND I TOLD YOU I WOULD GET IT
PUT THAT ON GOD
WASNT ME
I JUST SET THIS HOUSE ON FIRE ; SHHH DONT TELL MY PARENTS
THIS GOD TOLD ME TO
AND SO IT BEGINS
I DIDNT LIKE WHERE WE MOVED ; SO I LEFT THE STOVE ON WHILE WE WENT TO CHINA
MOMMY DIDNT GIVE ME A HAPPY MEAL ; SO I BURNED HER HOUSE DOWN
MY DOG DID SOMETHING BAD AND DIDNT GET IN TROUBLE ; NEITHER DID I
WHEN MOMMY DOES NOT LET ME WATCH TV
I HATE MY ROOM SO
BTS ; BT ARMY
MISSION COMPLETED
WHEN SOMEONE USES A ; TOILET
MY LIFE AS SCHOOL FAMILY AND MY CAREER KILL ME INTERNALLY
THE DIET I WAS GOING TO START 

WHEN YOU GET A NEW HAIR CUT ; NOW I CAN ATTRACT THE LADIES
YOU IN A BATTLE WITH ; YOUR FRIEND FOR FOOD
NOT SURE IF A SPINAL TAP HURTS ; OR IM USED TO IT
Worst loss Not sure if Cleveland Browns 2019 week one game or the 2018 trolls at the RedHead Wine Facebook page
NOT SURE I FOUND GOING TO SCHOOL IS ; OPTIONAL OR SOCIAL SUICIDE
IF THE EARTH IS FLAT HOW ; COME MY LIFE IS GOING DOWN HILL
I WANT TO BUY A SHAMPOO ; BUT IT SAYS DANDRUFF FREE
AND HERE I THOUGHT ; YOU LOVED ME
WAIT CHOCOLATE MILK DOESNT COME FROM BROWN COWS
HEY BOOMER
POST A FUN MEME ; OR CURE CANCER
EVERYTHING ON TV IS MAKE BELIEVE YES ALL OF IT
WHY THE HECK YOU ; MOVING US TO RESERVATIONS
WHEN YOU KNOW YOUR GOING ; TO TOYS R US
WHEN U TRY TO FIND ; THE RIGHT SAUCE FOR DINNER
THE FACE YOU MAKE ; WHEN THE SPANISH TEACHER SAYS FIVE SENTENCES FULLY IN SPANISH
NOW DONT TELL ME THAT YOU HAVE GOT A GIRLFRIEND
WHEN YOU GET A TRIPPING CALL ; IN FOOTBALL
WHEN PEOPLE SAY MERRY CHRISTMAS ; ON THE 21TH
GET OUT OF MY HOUSE ; BEFORE I KIC

ONE DOES NOT SIMPLY SMOKE ; AN EIGHTH OF WEED IN ONE SITTING
ONE DOES NOT SIMPLY ; END WORLD POVERTY
ONE DOES NOT SIMPLY ; SEE A MEME
WHEN U CALL U DUMB ; THAT AUTISTIC KID DOES NOT APPLY
YOU DO NOT JUST MAKE IT TO THE FRONT PAGE
ONE DOES NOT SIMPLY ; GET THREE BLUE RIBBONS ON STUDY ISLAND
ONE DOES NOT SIMPLY ; GO TO THE KITCHEN WITHOUT GETTING FOOD
ONE DOES NOT SIMPLY FIND A BOSTON TERRIER A BOSTON TERRIER FINDS YOU
ONE DOES NOT SIMPLY ; EXPECT THEIR PLAYERS TO STARE AT A BLACK HOLE FOR HOURS
ONE DOES NOT SIMPLY ; RAID AREA 51 WITHOUT NARUTO RUNNING
ONE DOES NOT SIMPLY ; EAT BRUSSEL SPROUTS WITHOUT BARKING
ONE DOES NOT SIMPLY ; GO TO THE BATHROOM WITHOUT OUT THEIR PHONE OR COMPUTER
ONE DOES NOT SIMPLY ; SAY ONE DOES NOT SIMPLY SAY ONE DOES NOT SIMPLY SAY ONE DOES NOT SIMPLY SAY ONE DOES NOT SIMPLY SAY ONE DOES NOT SIMPLY SAY ONE DOES NOT SIMPLY SAY
ONE DOES SIMPLY ; KILL THEMSELVES AND NOT DIE
ONE DOES NOT SIMPLY ; AREA 51 IS HARD TO ENTER
YOU DO NOT SIMPLY ; LET YOUR STEPSON BE HAPPY

In [11]:
words_embs = dict()

with open('glove.6B.300d.txt', 'rb') as f:
    for l in f:
        line = l.decode().split()
        word = line[0]
        if word in vocab.all_characters:
            words_embs[vocab.char_to_id[word]] = torch.FloatTensor(np.array(line[1:]).astype(np.float))

In [ ]:
model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True)
model.eval()
model.fc = torch.nn.Sequential()

In [12]:
#encoder
class EncoderRNN1(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN1, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(input_size, hidden_size * 2, n_layers, batch_first=False)
        self.lin = nn.Linear(2048 + 300, hidden_size)
        self.cnn = model
    def forward(self, input_pair, hidden):
        img = input_pair[1]
        word_inputs = input_pair[0]
        embedded = words_embs[word_inputs[0].item()]
        seq_len = len(word_inputs)
        for i in range(1, len(word_inputs)):
            embedded = words_embs[word_inputs[i].item()] + embedded
        embedded = embedded / seq_len
        with torch.no_grad():
            img = model(img)
        embedded = torch.cat((img.squeeze(), embedded), 0)
        embedded = embedded.view(1, 1, -1)
        hidden = self.lin(embedded)
        return  hidden

    def init_hidden(self):
        hidden = Variable(torch.zeros(self.n_layers, 1, self.hidden_size))
        return hidden

In [13]:
import torch.nn.functional as F

In [14]:
class Attn():
    pass

In [15]:
#decoder
class AttnDecoderRNN1(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1):
        super(AttnDecoderRNN1, self).__init__()
        
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size , n_layers, batch_first=False)
        self.out = nn.Linear(hidden_size , output_size)
    
    
    def forward(self, word_input,  last_hidden, l_c):
        
        global words_embs
        if (len(words_embs.get(word_input.item(), torch.LongTensor([0]))) != 1):
            word_embedded =words_embs[word_input.item()].view(1, 1, -1) # S=1 x B x N
        else:
            word_embedded = self.embedding(word_input).view(1, 1, -1)
        
        rnn_output, (hidden,c) = self.lstm(word_embedded, (last_hidden, l_c))

        m = nn.Dropout(p=self.dropout_p)
        rnn_output = rnn_output.squeeze(0) # S=1 x B x N -> B x N
        rnn_output = m(rnn_output)
        output = F.log_softmax(self.out(rnn_output))
        
        return output, (hidden, c)

In [ ]:
def create_pair(line,vocab):
    ans = list()
    ans1 = list()
    line11 = line[1]
    ans.append(vocab.char_to_id['<go>'])
    for k in line11.split(' '):
        ans.append(vocab.char_to_id[k.lower()])
    line1 = desc[line[0]][0]
    img = desc[line[0]][1]
    for k in line1.split(' '):
        ans1.append(vocab.char_to_id[k.lower()])
    ans.append(vocab.char_to_id['<eos>'])
    ans =torch.LongTensor(ans)
    ans1 = torch.LongTensor(ans1)

    return ((ans1,img), ans)

In [16]:
with open('model.pickle', 'rb') as f:
    model =  pickle.load(f) 

In [17]:
hidden_size = 300
n_layers = 1
dropout_p = 0

encoder = EncoderRNN1(vocab.size, hidden_size, n_layers)
decoder = AttnDecoderRNN1(hidden_size, vocab.size, n_layers, dropout_p=dropout_p)


learning_rate = 0.0005
encoder_optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate, weight_decay = 1e-5)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=learning_rate,weight_decay = 1e-5)
criterion = nn.NLLLoss()

In [ ]:
teacher_forcing_ratio = 0.75
clip = 5.0

In [ ]:
#training
def train1(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length, vocab):

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    loss = 0 

    input_length = input_variable[0].size()[0]
    target_length = target_variable.size()[0]

    encoder_hidden = encoder.init_hidden()
    encoder_hidden = encoder(input_variable, encoder_hidden)
    
    decoder_input = torch.Tensor([vocab.char_to_id['<go>']]).long()
    decoder_context = Variable(torch.zeros(1, decoder.hidden_size))
    decoder_hidden = encoder_hidden
    dec_c = Variable(torch.zeros(1, decoder.hidden_size )).view(1, 1, -1)
    use_teacher_forcing = random.random() < teacher_forcing_ratio
    if use_teacher_forcing:
        for di in range(target_length):
            decoder_output, (decoder_hidden,dec_c) = decoder(decoder_input,  decoder_hidden,dec_c)
            loss += criterion(decoder_output, target_variable[di].unsqueeze(0))
            decoder_input = target_variable[di]
    else:
        for di in range(target_length):
            decoder_output, (decoder_hidden,dec_c) = decoder(decoder_input,  decoder_hidden,dec_c)
            loss += criterion(decoder_output, target_variable[di].unsqueeze(0))
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]
            decoder_input = Variable(torch.LongTensor([[ni]]))
            if ni == vocab.char_to_id['<eos>']: break
            decoder_input = Variable(torch.LongTensor([[ni]])) # Chosen word is next input)
    loss.backward()
    torch.nn.utils.clip_grad_norm(encoder.parameters(), clip)
    torch.nn.utils.clip_grad_norm(decoder.parameters(), clip)
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item()  / target_length

In [ ]:
from copy import deepcopy
import random

import time
n_epochs = 30000000
plot_every = 50
print_every = 50

start = time.time()
plot_losses = []
print_loss_total = 0 
plot_loss_total = 0 

In [ ]:
#training
from tqdm import tqdm
for epoch in range(1, n_epochs + 1):
    
    training_pair = create_pair(random.choice(lines), vocab)
    input_variable = training_pair[0]
    target_variable = training_pair[1]

    loss = train1(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion,1000, vocab)
    print_loss_total += loss
    plot_loss_total += loss

    if epoch == 0: continue
    if epoch % 55555 == 0:
        if teacher_forcing_ratio > 0.5:
            teacher_forcing_ratio -= 0.1
    if epoch % print_every == 0:
        print_loss_avg = print_loss_total / print_every
        print_loss_total = 0
        print_summary = ' (%d %d%%) %.4f' % ( epoch, epoch / n_epochs * 100, print_loss_avg)
        print(print_summary)

    if epoch % plot_every == 0:
        plot_loss_avg = plot_loss_total / plot_every
        plot_losses.append(plot_loss_avg)
        plot_loss_total = 0

In [18]:
import pickle 
with open('enctopREALTOPnew3.pickle', 'rb') as f:
    encoder = pickle.load(f)
with open('dectopREALTOPnew3.pickle', 'rb') as f:
    decoder= pickle.load(f)
with open('vocabREALTOPnew3.pickle', 'rb') as f:
    vocab = pickle.load(f)
with open('decoptREALTOPnew3.pickle', 'rb') as f:
    decoder_optimizer = pickle.load(f)
with open('encopttREALTOPnew3.pickle', 'rb') as f:
    encoder_optimizer = pickle.load( f)
with open('datasetsREALTOPnew3.pickle', 'rb') as f:
    datasets =  pickle.load(f)
with open('datasetsREALTOPnew3.pickle', 'rb') as f:
    datasets =  pickle.load(f)
with open('linesREALTOPnew3.pickle', 'rb') as f:
    lines =  pickle.load(f) 
with open('model.pickle', 'rb') as f:
    model =  pickle.load(f) 

In [19]:
def gen(pic_id, encoder, decoder, encoder_optimizer, decoder_optimizer, max_length, vocab):
    ans = list()

    line1 = desc[pic_id][0]
    for k in line1.split(' '):
        ans.append(vocab.char_to_id[k.lower()])
    ans =torch.LongTensor(ans)
    img = desc[pic_id][1]

    encoder_hidden = encoder.init_hidden()
    encoder_hidden = encoder((ans,img), encoder_hidden)
    
    decoder_input = torch.Tensor([vocab.char_to_id['<go>']]).long()
    dec_c = Variable(torch.zeros(1, decoder.hidden_size )).view(1, 1, -1)
    decoder_hidden =  encoder_hidden                                                          
    result = str()
    for di in range(1000):
        decoder_output, (decoder_hidden,dec_c) = decoder(decoder_input,  decoder_hidden,dec_c)
        hh = decoder_output[0].clone().detach().numpy()
        sampler = torch.distributions.Categorical(torch.FloatTensor(np.exp(hh)))
        ni = sampler.sample().data.numpy().item()
        decoder_input = Variable(torch.LongTensor([[ni]]))
        if ni == vocab.char_to_id['<eos>']: break
        decoder_input = Variable(torch.LongTensor([[ni]])) 
        result += (vocab.id_to_char[ni]+' ')
    print(result[5:].upper())

In [22]:
pic_id = 4
gen(pic_id, encoder, decoder, encoder_optimizer, decoder_optimizer, 1000, vocab)

IF LIVE COULD STOP MAKING WIMP ; SKIP STUPID 


/home/rodion/miniconda3/envs/sphere-py37/lib/python3.7/site-packages/ipykernel_launcher.py:29: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
